<a href="https://colab.research.google.com/github/Angel-Diaz-H/scripts-python-sql/blob/main/sql_scripts_respaldos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import re

archivo_entrada = "/content/file3.sql"
archivo_salida = "backup.sql"

#leer archivo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# Dividir el archivo en bloques por cada sentencia que termina con ';'
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

selects_generados = []

for sentencia in sentencias:
    sentencia = sentencia.strip()
    #ignorar y saltar comentarios
    if not sentencia or sentencia.startswith("--"):
        continue

    #delete
    if sentencia.upper().startswith("DELETE FROM"):
        #'DELETE' por 'SELECT *'
        linea_select = re.sub(r"DELETE FROM\s+", "SELECT * FROM ", sentencia, flags=re.IGNORECASE)
        selects_generados.append(linea_select + ";")  #Agregar ;

    #UPDATE
    elif sentencia.upper().startswith("UPDATE"):
        #search tabla después de UPDATE
        tabla_match = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)
            #WHERE (en todo el bloque)
            where_idx = sentencia.upper().find("WHERE")
            if where_idx != -1:
                condicion = sentencia[where_idx:]
                linea_select = f"SELECT * FROM {tabla} {condicion}"
                selects_generados.append(linea_select + ";")

#save
with open(archivo_salida, "w") as f:
    for sel in selects_generados:
        f.write(sel + "\n")

print(f"{len(selects_generados)} SELECTs respaldados en '{archivo_salida}' 🎉")


✅ ¡Listo! Se generaron 15 SELECTs respaldados en 'scripts_SELECT_generados.sql' 🎉


In [2]:
import os

nombre_archivo = "file1.v2.sql"
ubicacion_archivo = os.path.abspath(nombre_archivo)

print(f"La ubicación del archivo '{nombre_archivo}' es: {ubicacion_archivo}")

La ubicación del archivo 'file1.v2.sql' es: /content/file1.v2.sql
